In [707]:
import numpy as np

In [708]:
def phase1(matrix, rhs, z, numxvars):
    '''Simplex algorithm to solve linear programming problems
    
    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side
    
    rhs: numpy ndarray
        Right-hand side vector
    
    numxvars: int
        Number of x variables
        
    direction: {+1 , -1}
        For maximization problems use +1 and for minimization problems use -1 instead.
    '''
    print("="*30, "PHASE I", "="*30, "\n")
    
    
    num_rows, num_cols = matrix.shape
    onecols = np.where(matrix == 1)[1]
    cb_index = onecols[onecols >= numxvars]
    
    w = np.where(np.abs(z) >= 1000 , 1, 0)  # new objective function
    cb = w[cb_index]                  
    zj = cb.dot(matrix)
    net_evaluation = zj - w
    
    solutions = []
    zvalues = []
    
    iteration = 0
    status = True
    while np.any(net_evaluation > 0):
        solution = np.zeros_like(z)
        entering = net_evaluation.argmax()  # entering variables (index)

        key_col = matrix[ : , entering]
        ratios = np.divide(rhs, key_col, out=np.full_like(rhs, np.inf), where=key_col>0)
        leaving = ratios.argmin()   # leaving variables (index)
        pivot = matrix[leaving, entering]
        
        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot
        
        for i in range(num_rows):
            if i == leaving:
                continue
            factor = matrix[i, entering]
            matrix[i] = -factor * matrix[leaving] + matrix[i]
            rhs[i] = -factor * rhs[leaving] + rhs[i]
        
        cb_index[leaving] = entering
        cb = w[cb_index]
        
        zj = cb.dot(matrix)
        
        net_evaluation = zj - w

        solution[cb_index] = rhs  # basics
        
        iteration += 1
        print(f"Iteration {iteration}")
        print(matrix,  "\n")
        print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}", "\n")
        
        zvalues.append(cb.dot(rhs))
        solutions.append(solution)
        
    if cb.dot(rhs) > 0: 
        status = False  # non-optimal
    return cb_index, solutions, zvalues, iteration, status

In [709]:
def phase2(matrix, rhs, z, numxvars, cb_index, solutions, zvalues, iteration, direction):
# def phase2(z, direction=1):
    '''Simplex algorithm to solve linear programming problems
    
    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side
    
    rhs: numpy ndarray
        Right-hand side vector
    
    numxvars: int
        Number of x variables
        
    direction: {+1 , -1}
        For maximization problems use +1 and for minimization problems use -1 instead.
    '''
    print("="*30, "PHASE II", "="*30, "\n")
    
    num_rows = len(matrix)
    
    cb = z[cb_index]
    zj = cb.dot(matrix)
    
    net_evaluation = direction * (z - zj)
    
    solution = np.zeros_like(z)
    solution[cb_index] = rhs
    
    zvalues[-1] = cb.dot(rhs)
    if np.all(net_evaluation <= 0):
            print(solution, cb.dot(rhs), "\n")
            print(f"Optimal solution found in {iteration} iterations" )
            
    while np.any(net_evaluation > 0):
        solution = np.zeros_like(z)
        entering = net_evaluation.argmax()  # entering variables (index)

        key_col = matrix[ : , entering]
        ratios = np.divide(rhs, key_col, out=np.full_like(rhs, np.inf), where=key_col>0)
        leaving = ratios.argmin()   # leaving variables (index)
        pivot = matrix[leaving, entering]
        
        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot
        
        for i in range(num_rows):
            if i == leaving:
                continue
            factor = matrix[i, entering]
            matrix[i] = -factor * matrix[leaving] + matrix[i]
            rhs[i] = -factor * rhs[leaving] + rhs[i]
        
        cb_index[leaving] = entering
        cb = cj[cb_index]
        
        zj = cb.dot(matrix)
        
        net_evaluation = direction * (z - zj)

        solution[cb_index] = rhs  # basics
        
        iteration += 1
        print(f"Iteration {iteration}")
        print(matrix,  "\n")
        zvalues.append(cb.dot(rhs))
        print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}", "\n")
        
        solutions.append(solution)
        if np.all(net_evaluation <= 0):
            print(f"Optimal solution found in {iteration} iterations")
    return np.array(solutions), zvalues

In [710]:
def twophase(matrix, rhs, z, numxvars, direction=1):
    '''Simplex algorithm to solve linear programming problems
    
    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side
    
    rhs: numpy ndarray
        Right-hand side vector
    
    numxvars: int
        Number of x variables
        
    direction: {+1 , -1}
        For maximization problems use +1 and for minimization problems use -1 instead.
    '''
    cbind, sols, values, itr, is_optimal = phase1(matrix, rhs, z, numxvars)
    if is_optimal:
        return phase2(matrix, rhs,  z, numxvars, cb_index=cbind, solutions=sols, zvalues=values, iteration=itr, direction=direction)
    else:
        print("INFEASIBLE")


In [711]:
cj = np.array( [12, 20, 0, 0, 1000, 1000]  ,  dtype=float)

A = np.array( [
    [6,  8, -1,  0, 1, 0],  # 0
    [7, 12,  0, -1, 0, 1],  # 1
] , dtype=float)

b = np.array( [100, 120] , dtype=float )

In [712]:
sols, zvalues = twophase(matrix=A, rhs=b, z=cj, numxvars=2, direction=-1)
print()
print(sols)
print(zvalues)

============================== PHASE I ============================== 

Iteration 1
[[ 1.33333333  0.         -1.          0.66666667  1.         -0.66666667]
 [ 0.58333333  1.          0.         -0.08333333  0.          0.08333333]] 

Solution [ 0. 10.  0.  0. 20.  0.] 	Z: 20.00 

Iteration 2
[[ 1.      0.     -0.75    0.5     0.75   -0.5   ]
 [ 0.      1.      0.4375 -0.375  -0.4375  0.375 ]] 

Solution [15.    1.25  0.    0.    0.    0.  ] 	Z: 0.00 

============================== PHASE II ============================== 

[15.    1.25  0.    0.    0.    0.  ] 205.00000000000003 

Optimal solution found in 2 iterations

[[ 0.   10.    0.    0.   20.    0.  ]
 [15.    1.25  0.    0.    0.    0.  ]]
[20.0, 205.00000000000003]


In [713]:
cj = np.array([2,3,0,0,0,0,0,-1000,-1000], dtype=float)

A = np.array([
    [1,  1, 1,  0, 0,  0, 0,  0, 0],
    [0,  1, 0, -1, 0,  0, 0,  1, 0],
    [0,  1, 0,  0, 1,  0, 0,  0, 0],
    [1, -1, 0,  0, 0, -1, 0,  0, 1],
    [1,  0, 0,  0, 0,  0, 1,  0, 0],
    ]
    , dtype=float)

b = np.array([30, 3, 12, 0, 20], dtype=float)

In [714]:
sols, zvalues = twophase(matrix=A, rhs=b, z=cj, numxvars=2, direction=1)
print()
print(sols)
print(zvalues)

============================== PHASE I ============================== 

Iteration 1
[[ 0.  2.  1.  0.  0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 1. -1.  0.  0.  0. -1.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  1.  1.  0. -1.]] 

Solution [ 0.  0. 30.  0. 12.  0. 20.  3.  0.] 	Z: 3.00 

Iteration 2
[[ 0.  0.  1.  2.  0.  1.  0. -2. -1.]
 [ 0.  1.  0. -1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  1.  0.  0. -1.  0.]
 [ 1.  0.  0. -1.  0. -1.  0.  1.  1.]
 [ 0.  0.  0.  1.  0.  1.  1. -1. -1.]] 

Solution [ 3.  3. 24.  0.  9.  0. 17.  0.  0.] 	Z: 0.00 

============================== PHASE II ============================== 

Iteration 3
[[ 0.  0.  1.  0. -2.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.  0. -1.  0.]
 [ 1.  0.  0.  0.  1. -1.  0.  0.  1.]
 [ 0.  0.  0.  0. -1.  1.  1.  0. -1.]] 

Solution [12. 12.  6.  9.  0.  0.  8.  0.  0.] 	Z: 60.00 

Iteration 4
[[ 0.  0.  1.  0. -2.  1.  0.  0. -1.]